In [42]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set_style("darkgrid")
mpl.rcParams['figure.figsize'] = (20,5)

Data Cleansing

In [44]:
ggc_sales = pd.read_csv('GGC items raw.csv')
print(ggc_sales.head())
print(ggc_sales.shape)

       Date      Time Time Zone  Category               Item  Qty  \
0  7/7/2024  11:36:43   Arizona  Beverage         Mad Muscle    1   
1  7/7/2024  11:03:27   Arizona       NaN           Chai Tea    1   
2  7/7/2024  11:03:27   Arizona       NaN        pink elixir    1   
3  7/7/2024  11:03:27   Arizona       NaN  caramel cold brew    1   
4  7/7/2024  11:01:01   Arizona       NaN  caramel cold brew    1   

  Price Point Name  SKU   Modifiers Applied Gross Sales  ... Event Type  \
0          Regular  NaN                 NaN      $6.50   ...    Payment   
1          Regular  NaN         Large, Iced      $5.50   ...    Payment   
2          Regular  NaN               Large      $6.00   ...    Payment   
3          Regular  NaN               Large      $6.00   ...    Payment   
4          Regular  NaN  Small, Almond Milk      $5.50   ...    Payment   

                     Location Dining Option                 Customer ID  \
0  A Lovely Mess Company, LLC      For Here                

In [45]:
# Check for columns with all missing values.
print(ggc_sales.isnull().sum())

Date                        0
Time                        0
Time Zone                   0
Category                 1839
Item                        0
Qty                         0
Price Point Name          108
SKU                      3373
Modifiers Applied         396
Gross Sales                 0
Discounts                   0
Net Sales                   0
Tax                         0
Transaction ID              0
Payment ID                  0
Device Name               112
Notes                    3328
Event Type                  0
Location                    0
Dining Option             112
Customer ID              1609
Customer Reference ID    3373
Unit                        0
Count                       0
Itemization Type          108
Fulfillment Note         3373
Token                     108
dtype: int64


In [46]:
# Drop columns with all missing values. Drop columns with unnecessary information for analysis.
# Fill missing values with placeholder text.
ggc_sales.drop(['SKU', 'Customer Reference ID', 'Fulfillment Note'], axis='columns', inplace=True)
ggc_sales.drop(['Time Zone', 'Device Name', 'Details', 'Location', 'Dining Option', 'Unit', 'Count', 'Token'], axis='columns', inplace=True)
ggc_sales.fillna('NA', inplace=True)
print(ggc_sales.info())

KeyError: "['Details'] not found in axis"

In [ ]:
# Adjust column names to a more accessible format.
ggc_sales.columns = ggc_sales.columns.str.replace(' ', '_').str.lower()
print(ggc_sales.columns)

In [ ]:
# Create a datetime column comining the date and time values provided.
ggc_sales['datetime'] = pd.to_datetime(ggc_sales['date'].astype(str) + ' ' + ggc_sales['time'].astype(str))
# Convert the Date column to datetime format
ggc_sales['date'] = pd.to_datetime(ggc_sales['date'])
# Convert Qty column to int
ggc_sales['qty'] = ggc_sales.qty.astype('int64')
# Convert values in currency columns from string to float.
ggc_sales[['gross_sales', 'tax', 'net_sales', 'discounts']] = ggc_sales[['gross_sales', 'tax', 'net_sales', 'discounts']].replace({'\$': '', ',': ''}, regex=True).astype('float')
# Create a column adding tax to the net sale, giving a total for each transaction.
ggc_sales['final_sale'] = round(ggc_sales.net_sales + ggc_sales.tax, 2)
print(ggc_sales.head())

In [ ]:
# Divide the modifiers_added column into multiple columns for each modifier.
# Assign lists for each modifier.
sizes = ['Small', 'Large']
temperatures = ['Hot', 'Iced', 'Blended']
milks = ['Whole Milk', 'Skim Milk', 'Almond Milk', 'Oat Milk', 'Half & Half', 'Heavy Whipping Cream', 'Lemonade']
flavors = ['Brown Sugar Cinnamon', 'Caramel', 'Hazelnut', 'Irish Cream', 'Mocha', 'Peppermint', 'SF Peppermint', 'Strawberry',\
           'Sugar Free Blueberry', 'Sugar Free Caramel', 'Sugar Free Vanilla', 'Vanilla', 'White Chocolate', 'Black Cherry', 'Blue Raspberry',\
           'Lavender', 'Peach', 'Pumpkin', 'Raspberry', 'SF Caramel', 'Watermelon']
addons = ['Add Shot', 'Cold Foam', 'Double Shot', 'Extra Sauce', 'No Whip', 'Whipped Cream']

# Create separate columns for each category
ggc_sales['size'] = None
ggc_sales['temperature'] = None
ggc_sales['milk'] = None
ggc_sales['flavors'] = None
ggc_sales['addons'] = None

# Function to assign values to the correct columns
def assign_modifiers(row):
    modifiers = row['modifiers_applied'].split(', ')
    size = None
    temperature = None
    milk = None
    flavor_list = []
    addon_list = []
    
    for mod in modifiers:
        if mod in sizes:
            size = mod
        elif mod in temperatures:
            temperature = mod
        elif mod in milks:
            milk = mod
        elif mod in flavors:
            flavor_list.append(mod)
        elif mod in addons:
            addon_list.append(mod)
    
    row['size'] = size
    row['temperature'] = temperature
    row['milk'] = milk
    row['flavors'] = ', '.join(flavor_list) if flavor_list else None
    row['addons'] = ', '.join(addon_list) if addon_list else None
    
    return row

# Apply the function to each row, and drop the original column
ggc_sales = ggc_sales.apply(assign_modifiers, axis=1)
ggc_sales.drop('modifiers_applied', axis='columns', inplace=True)
print(ggc_sales.head())

In [ ]:
# Combine same drinks under different names
ggc_sales['item'] = ggc_sales['item'].replace({
    'Iced Green Tea': 'Green Tea',
    'iced green tea peach lemonade': 'green tea peach lemonade',
    'Latte2': 'Latte',
    'the chocoholic frapp': 'chocoholic frapp'
})

print(ggc_sales.item.unique())

In [ ]:
# Update values for price point name and category
foods = ['cookie', 'cupcake', 'loaf', 'muffin', 'pastry', 'scone', 'string cheese']
coffee_teas = ['mad muscle', 'chai tea', 'caramel cold brew', 'americano', 'dirty filthy little chai', 'matcha latte',\
               'green tea peach lemonade', 'macchiato', 'cold brew', 'frapp', 'matcha monk', 'green tea', 'shaken espresso',\
               'coffee', 'doppio', 'pumpkin spice latte', 'latte', 'chocoholic frapp', 'pumpkin chai', 'cappuccino',\
               'irish goodbye', 'tsukuyomi', 'mad muscle iced latte', 'the chocoholic frapp', 'coffee frapp',\
               "sugar we're going down", 'drip coffee', 'dalgona style', 'tea', 'flat white']
beverages = ['black elixir', 'cream frapp', 'elixir', 'flavored lemonade', 'frozen elixir', 'frozen lemonade',\
             'hot chocolate', 'ice wizard', 'kids milk', 'mana potion', 'pink elixir', 'rasengan']

for i, item in enumerate(ggc_sales.item):
    item_l = item.lower()
    if item == 'Custom Amount':
        ggc_sales.at[i, 'price_point_name'] = 'Custom'
        ggc_sales.at[i, 'category'] = 'Custom'
    elif item in ['Small Event Drink', 'Large Event Drink', 'Event Drink', 'Event Drink (smalls) (full menu)', 'Special Event']:
        ggc_sales.at[i, 'price_point_name'] = 'Events'
        ggc_sales.at[i, 'category'] = 'Events'
    else :
        ggc_sales.at[i, 'price_point_name'] = 'Regular'
        if any(keyword in item_l for keyword in foods):
            ggc_sales.at[i, 'category'] = 'Food'
        elif any(keyword in item_l for keyword in beverages):
            ggc_sales.at[i, 'category'] = 'Beverages'
        elif any(keyword in item_l for keyword in coffee_teas):
            ggc_sales.at[i, 'category'] = 'Coffee and Tea'
        else:
            ggc_sales.at[i, 'category'] = 'Unknown'

print(ggc_sales.price_point_name.unique())
print(ggc_sales.category.unique())

In [ ]:
# Assign temperature values to drinks with only one temperature option
hot_drinks = ['hot chocolate']
iced_drinks = ['caramel cold brew', 'green tea peach lemonade', 'cold brew', 'shaken espresso', 'mad muscle iced latte',\
               "sugar we're going down", 'black elixir', 'elixir', 'flavored lemonade', 'kids milk', 'pink elixir']
blended = ['mad muscle', 'frapp', 'chocoholic frapp', 'tsukuyomi', 'coffee frapp', 'cream frapp',\
           'frozen elixir', 'frozen lemonade', 'ice wizard', 'mana potion', 'rasengan']

for i, item in enumerate(ggc_sales.item):
    item_l = item.lower()
    if any(keyword in item_l for keyword in hot_drinks):
        ggc_sales.at[i, 'temperature'] = 'Hot'
    elif any(keyword in item_l for keyword in iced_drinks):
        ggc_sales.at[i, 'temperature'] = 'Iced'
    elif any(keyword in item_l for keyword in blended):
        ggc_sales.at[i, 'temperature'] = 'Blended'

print(ggc_sales[ggc_sales['price_point_name'] == 'Regular'].head())

In [ ]:
# Check for drinks with missing size values
print(ggc_sales['item'][(~ggc_sales['category'].isin(['Food','Events', 'Custom'])) & (ggc_sales['size'].isna())].unique())

In [ ]:
# Replace missing size values for Mad Muscle drinks, which are always large
for i, item in enumerate(ggc_sales.item) :
    if item in ['Mad Muscle', 'Mad Muscle Iced Latte']:
        ggc_sales.at[i, 'size'] = 'Large'
# Replace missing size values for Matcha Monk based on drink price
size_price = {
    4.50: 'Small',
    5.00: 'Small',
    5.50: 'Large',
    6.00: 'Large'
}

for i, row in ggc_sales.iterrows():
    if row['item'] == 'Matcha Monk' and pd.isna(row['size']):
        ggc_sales.at[i, 'size'] = size_price.get(row['gross_sales'], 'Unknown')

print(ggc_sales['size'][ggc_sales['item'].isin(['Mad Muscle', 'Mad Muscle Iced Latte', 'Matcha Monk'])].unique())

In [ ]:
# Adjust 'Whole Milk' to '2% Milk'
ggc_sales['milk'] = ggc_sales['milk'].replace({'Whole Milk': '2% Milk', 'Lemonade': None})

# Assign milk values for drinks with known milks
has_milk = ['latte', 'dirty filthy little chai', 'matcha latte', 'macchiato', 'matcha monk', 'shaken espresso', 'hot chocolate', 'kids milk',\
            'pumpkin spice latte', 'pumpkin chai', 'cappuccino', 'irish goodbye', "sugar we're going down", 'dalgona style', 'flat white']
has_hwc = ['mad muscle', 'frapp', 'chocoholic frapp', 'tsukuyomi', 'coffee frapp', 'cream frapp', 'rasengan', 'tsukuyomi']

for i, item in enumerate(ggc_sales['item']):
    item_l = item.lower()
    if pd.isna(ggc_sales.at[i, 'milk']):
        if any(drink in item_l for drink in has_milk):
            ggc_sales.at[i, 'milk'] = '2% Milk'
        elif any(drink in item_l for drink in has_hwc):
            ggc_sales.at[i, 'milk'] = 'Heavy Whipping Cream'
# Remove outlier
ggc_sales.loc[2951, 'milk'] = None

print(ggc_sales.milk.unique())

ANALYSIS

Saving

In [ ]:
# Filter for tax = 0
ggc_sales[ggc_sales['tax'] == 0]

In [ ]:
# Determine sales tax percentage
has_tax = ggc_sales['tax'][ggc_sales['tax'] != 0]
reg_tax = has_tax[ggc_sales['price_point_name'] == 'Regular']
reg_net = ggc_sales['net_sales'][ggc_sales['price_point_name'] == 'Regular']
tax_pct = (reg_tax / reg_net).mean()
tax_pct

In [ ]:
# Analyze losses from inaccurate tax amounts
no_tax = ggc_sales['net_sales'][ggc_sales['tax'] == 0]
missing_tax = no_tax * tax_pct
tax_loss = no_tax + missing_tax
tax_loss

In [ ]:
ggc_sales['est_tax'] = round(ggc_sales['net_sales'] * tax_pct, 2)
ggc_sales['est_final'] = round(ggc_sales['net_sales'] + ggc_sales['est_tax'], 2)
ggc_sales['est_final']

In [ ]:
tax_final = ggc_sales['final_sale'][ggc_sales['tax'] == 0]

tax_compare = pd.DataFrame(columns= ['original_sale', 'est_tax_sale'])
tax_compare['original_sale'] = tax_final
tax_compare['est_tax_sale'] = tax_loss
tax_compare

In [ ]:
# Calculate the sums of the two columns
sum_original_sale = tax_compare['original_sale'].sum()
sum_est_tax_sale = tax_compare['est_tax_sale'].sum()

# Create a DataFrame with these sums for easier plotting
sum_data = pd.DataFrame({
    'Sale Type': ['Original Revenue', 'Estimated Tax-Inclusive Revenue'],
    'Total Amount': [sum_original_sale, sum_est_tax_sale]
})

# Plotting the sums as a bar graph
plt.figure(figsize=(10, 6))
bars = plt.bar(sum_data['Sale Type'], sum_data['Total Amount'], color=['blue', 'green'])

# Adding labels and title
plt.xlabel('Sale Type')
plt.ylabel('Total Amount ($)')
plt.title('Total Amount Comparison: Original Revenue vs. Estimated Tax-Inclusive Revenue')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.02 * yval, f'{yval:.2f}', ha='center', va='bottom')

# Show the plot
plt.show()

Analysis of Drink Addons

In [ ]:
# Update flavors from notes
ggc_sales.loc[(ggc_sales['notes'].str.contains('Pink|PE')) & (ggc_sales['flavors'].isna()), 'flavors'] = 'Peach, Strawberry'
ggc_sales.loc[(ggc_sales['notes'] == 'Peach') & (ggc_sales['flavors'].isna()), 'flavors'] = 'Peach'
ggc_sales.loc[(ggc_sales['notes'].str.contains('Straw|Strawberry', case=False, na=False)) & (ggc_sales['flavors'].isna()), 'flavors'] = 'Strawberry'
ggc_sales.loc[(ggc_sales['notes'] == 'Carm') & (ggc_sales['flavors'].isna()), 'flavors'] = 'Caramel'
ggc_sales.loc[(ggc_sales['notes'].str.contains(r'free caramel')) & (ggc_sales['flavors'].isna()), 'flavors'] = 'Sugar Free Caramel'
ggc_sales.loc[(ggc_sales['notes'].str.contains('Blueberry')) & (ggc_sales['flavors'].isna()), 'flavors'] = 'Blueberry'

# Additional updates
ggc_sales.loc[(ggc_sales['notes'].str.contains('Shaken|SE', case=False, na=False)) & (ggc_sales['item'] == 'Latte'), 'item'] = 'Shaken Espresso'
ggc_sales.loc[(ggc_sales['notes'].str.contains(r'soft top', case=False, na=False)) & (ggc_sales['addons'].isna()), 'addons'] = 'Cold Foam'
ggc_sales.loc[(ggc_sales['notes'].str.contains('no cream', case=False, na=False)), 'milk'] = None
ggc_sales.loc[(ggc_sales['item'] == 'caramel cold brew') & (ggc_sales['flavors'].isna()), 'flavors'] = 'Caramel'

In [ ]:
# Create filtered DataFrame for drink analysis
ggc_reg = ggc_sales[ggc_sales['price_point_name'] == 'Regular']
ggc_drinks = ggc_reg[ggc_reg['category'].isin(['Beverages', 'Coffee and Tea'])]
drinks_table = ggc_drinks[['item', 'qty', 'size', 'milk', 'flavors', 'addons', 'net_sales', 'final_sale', 'notes']]
w_addons = drinks_table[~drinks_table['addons'].isna()]
w_notes = drinks_table[drinks_table['notes'] != 'NA']
w_notes

In [ ]:
# Analyze addon costs

Revenue Generating

Overall Popularity

In [ ]:
# Create function to plot the total sum of an aggregate value for each column value
def agg_plot(dataframe, col, agg_value, title, xlabel, ylabel):
    agg_group = dataframe.groupby(col)[agg_value].sum().sort_values(ascending=False)
    agg_group.plot(kind='bar')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

# Use agg_plot to plot the total quantity purchased for each drink
agg_plot(ggc_drinks, 'item', 'qty', 'Quantity Purchased by Drink', 'Drink Name', 'Qty Ordered')

Analysis by Drink Type

In [ ]:
# Ensure all drink names are lowercase for consistent comparison
ggc_sales['item'] = ggc_sales['item'].str.lower()

# Categorize drinks into new DataFrames
teas_df = ggc_sales[(ggc_sales['item'].str.contains('tea|chai|matcha'))]

frapps_df = ggc_sales[(ggc_sales['temperature'] == 'Blended') & (~ggc_sales['milk'].isna())]

elixirs_df = ggc_sales[ggc_sales['item'].str.contains('elixir|mana potion|ice wizard')]

# Print the unique items of each DataFrame for verification
print("Teas:", teas_df.item.unique())
print("Frapps:", frapps_df.item.unique())
print("Elixirs:", elixirs_df.item.unique())

In [ ]:
# Create a DataFrame to filter out event, custom, and food items
misc_df = ggc_sales[ggc_sales['category'].isin(['Custom', 'Events', 'Food'])]
# Create a combination of the previous categories
comb_df = pd.concat([teas_df, frapps_df, elixirs_df, misc_df])

# Anti join against the combo DataFrame to list items not yet categorized
merge_table = ggc_sales.merge(comb_df, on='item', how='left', indicator=True)
merge_item_list = merge_table.loc[merge_table['_merge'] == 'left_only', 'item']
anti_merge_df = ggc_sales[ggc_sales['item'].isin(merge_item_list)]
anti_merge_df.item.unique()

# Create the Coffees DataFrame by filtering out non-coffee items
coffees_df = anti_merge_df[~anti_merge_df['item'].str.contains('lemonade|kids milk|hot chocolate')]

# Print the unique items in the Coffees DataFrame to verify
coffees_df.item.unique()

In [ ]:
# Bar Chart analyzing the Coffees DataFrame by drink name
agg_plot(coffees_df, 'item', 'qty', 'Coffee Drink Overall Popularity', 'Drink Name', 'Qty Ordered')

In [ ]:
# Coffee Trend
import matplotlib.dates as mdates

# Create Function for plotting trends

def trend_plot(dataframe, col1, col2, agg_value, limit, title, xlabel, ylabel):
    # Group by selected columns, then sum the value chosen to aggregate
    trend_agg = dataframe.groupby([col1, col2])[agg_value].sum().reset_index()
    # Pivot the DataFrame to have the first column as rows and the second column as columns
    trend_pivot = trend_agg.pivot(index=col1, columns=col2, values=agg_value).fillna(0)
    # Calculate the total of the agg value
    total_qty = trend_pivot.sum(axis=0)
    # Limit the plot to only the top x, set by 'limit'
    plot_limit = total_qty.nlargest(limit).index
    filtered_pivot = trend_pivot[plot_limit]
    # Plot the data as a line chart
    for column in filtered_pivot.columns:
        plt.plot(filtered_pivot.index, filtered_pivot[column], label=column)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(title=col2)
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

# Use the trend_plot function to plot the top 5 Coffee drinks using the coffees_df dataframe

trend_plot(coffees_df, 'date', 'item', 'qty', 5, 'Coffee Drinks Popularity Trend (Top 5)', 'Date', 'Quantity Ordered')

In [ ]:
# Plot the overall popularity of tea drinks
agg_plot(teas_df, 'item', 'qty', 'Tea Drink Overall Popularity', 'Drink Name', 'Qty Ordered')

In [ ]:
# Tea Trend
trend_plot(teas_df, 'date', 'item', 'qty', 4, 'Tea Drinks Popularity Trend (Top 4)', 'Date', 'Quantity Ordered')

In [ ]:
# Frapps
agg_plot(frapps_df, 'item', 'qty', 'Frapp Drink Overall Popularity', 'Drink Name', 'Qty Ordered')

In [ ]:
#Frapp Trend
trend_plot(frapps_df, 'date', 'item', 'qty', 3, 'Frapp Drinks Popularity Trend (Top 3)', 'Date', 'Qty Ordered')

In [ ]:
# Elixirs
agg_plot(elixirs_df, 'item', 'qty', 'Elixir Drink Overall Popularity', 'Drink Name', 'Qty Ordered')

In [ ]:
#Elixir Trend
trend_plot(elixirs_df, 'date', 'item', 'qty', 4, 'Elixir Drinks Popularity Trend (Top 4)', 'Date', 'Qty Ordered')

In [ ]:
# Top 5 Trend
trend_plot(ggc_drinks, 'date', 'item', 'qty', 5, 'Top 5 Drinks Popularity Trend', 'Date', 'Qty Ordered')

In [ ]:
# Drinks ordered as Large vs Small
# Filter for relevant values
agg_size = ggc_drinks.groupby(['item', 'size'])['qty'].sum().reset_index()
agg_size['total_qty'] = agg_size.groupby('item')['qty'].transform('sum')
agg_size = agg_size.sort_values(by='total_qty', ascending=False)
pivot_size = agg_size.pivot_table(index='item', columns='size', values='qty').fillna(0)
pivot_size = pivot_size.loc[agg_size['item'].unique()]



# Plot axes in stacked bar
size_anlz = pivot_size.plot(kind='bar', stacked=True)
plt.title('Drink Popularity by Size (Large vs Small)')
plt.xlabel('Drink Name')
plt.ylabel('Qty of Drinks Ordered')

plt.show()

Flavor Analysis

In [ ]:
# Analyze popularity of flavors based on sum of Qty


In [ ]:
# Split the 'flavors' column into individual flavors
w_flavor = ggc_drinks[~ggc_drinks['flavors'].isna()]
flavors_expanded = w_flavor['flavors'].str.get_dummies(sep=', ')

# Add the quantity to each flavor
flavors_expanded = flavors_expanded.mul(ggc_drinks['qty'], axis=0)

# Plot the Qty Purchased for each flavor
pop_flavor = flavors_expanded.sum(axis=0).sort_values(ascending=False)
pop_flavor.plot(kind='bar')

In [ ]:
# Identify the top 5 flavors by total quantity ordered
top_5_flavors = pop_flavor.head(5).index

# Create a DataFrame to store the time series data
time_series_df = w_flavor[['date']].copy()

# Add the top 5 flavors quantities to the time series DataFrame
for flavor in top_5_flavors:
    time_series_df[flavor] = flavors_expanded[flavor]

# Group by date and sum the quantities
time_series_df = time_series_df.groupby('date').sum()

# Plot the trends for the top 5 flavors
for flavor in top_5_flavors:
    plt.plot(time_series_df.index, time_series_df[flavor], label=flavor)

plt.xlabel('Date')
plt.ylabel('Quantity Ordered')
plt.title('Trends for Top 5 Flavors Over Time')
plt.legend(title='Flavor')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

# Format the x-axis to show only the month
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.show()

Market Basket Analysis

In [ ]:
# Market basket analysis
# Filter relevant fields
market_b_df = ggc_reg[['transaction_id', 'customer_name', 'date', 'item', 'size', 'temperature', 'milk', 'flavors', 'addons', 'qty', 'final_sale']]

market_join = market_b_df.merge(market_b_df, left_on='transaction_id', right_on='transaction_id')
market_join

In [76]:
ggc_sales.to_csv('ggc sales cleaned.csv')